In [ ]:
import pandas as pd
import numpy as np

In [ ]:
almacen = pd.read_excel('Base Almacén - ITBA.xlsx', sheet_name='lx03 04112013')
estanterias_picking = almacen[almacen['Sector'] == 'Picking']['Ubicación']

In [ ]:
movimientos = pd.read_csv("Movimientos Pallet Completo.csv", encoding='ISO-8859-1', sep=';')
movimientos = movimientos[~((movimientos['Ubicproc'].isin(estanterias_picking)) | (movimientos['UbicDest'].isin(estanterias_picking)))]

In [ ]:
maestro = pd.read_excel('Mestro de Articulos - ITBA.xlsx', header=2)
mov_info = pd.merge(movimientos, maestro[['Cód. Material', 'Almacenamiento',	'Descr. Nível 1']], left_on='Material', right_on='Cód. Material', how='left')

In [ ]:
mov_info = mov_info.sort_values(by='Feconf', ascending=False)
mov_info = mov_info[mov_info['Mvto'].isin(['Recepción', 'Salidas'])]
mov_info["UbicGrouping"] = np.where(mov_info["Mvto"] == "Recepción", mov_info["Ubicproc"], mov_info["UbicDest"])
mov_info["id_viaje"] = mov_info.groupby(["Feconf","Usuario","UbicGrouping"]).ngroup()

In [ ]:
mov_info[["Material","Mvto","id_viaje","Hinicio","Feconf","Almacenamiento"]].rename(columns={"Mvto":"movimiento","Hinicio":"hora","Feconf":"fecha"}).sort_values(["id_viaje","fecha","hora"]).to_csv("movements.csv", index=False)